## Phase 2: Database Setup and Data Migration

**1. Executive Summary**

In this phase, we established the foundational infrastructure for our data mining project. Our objective was to migrate unstructured JSON data from the OpenFDA Drug Shortages API into a structured, relational PostgreSQL database.

We successfully designed a normalized database schema that handles the complex one-to-many relationships inherent in drug packaging data. We then implemented a robust ETL (Extract, Transform, Load) pipeline using Python, ensuring data integrity through automated cleaning, type conversion, and secure credential management.

**2. Database Design & Optimization**

**2.1 Technology Stack**

We selected **PostgreSQL** as our relational database management system (RDBMS). Its strict adherence to SQL standards and robust handling of complex queries make it ideal for the analytical workloads we will perform in Phase 4.

**2.2 Schema Architecture**

The raw data from OpenFDA presents a challenge: a single "shortage event" often affects multiple specific products (different dosages, package sizes, or NDCs).

To optimize for data integrity and reduce redundancy, we rejected a flat-file approach in favor of a relational design consisting of two tables:

- `drug_shortages` (Parent Table): Stores high-level information about the shortage event (Generic Name, Manufacturer, Reason, Status).

- `affected_products` (Child Table): Stores the granular details of the specific items affected (NDC, Presentation/Dosage).

**2.3 Normalization Strategy (2NF vs. 3NF)**

We engineered the schema to be effectively in Second Normal Form (2NF). This allows us to link multiple products to a single shortage ID via a Foreign Key, preventing the duplication of general shortage details.

- **Optimization Decision:** We intentionally stopped short of strict Third Normal Form (3NF). While the `presentation` field technically depends on the `ndc` (a transitive dependency), we chose to keep them together in the `affected_products` table. This "denormalized" approach minimizes expensive `JOIN` operations during our future analysis phase, optimizing read performance for data mining tasks.

In [5]:
# Schema Definition

sql_code = """
CREATE TABLE drug_shortages (
    shortage_id VARCHAR(255) PRIMARY KEY,
    drugname TEXT,
    manufacturer TEXT,
    status TEXT,
    shortagereason TEXT,
    availabilityscore FLOAT, -- Placeholder for Phase 3 Analysis
    regionaffected TEXT DEFAULT 'USA',
    lastupdate DATE
);

CREATE TABLE affected_products (
    id SERIAL PRIMARY KEY,
    shortage_id VARCHAR(255),
    ndc VARCHAR(50),
    presentation TEXT,
    status TEXT,
    FOREIGN KEY (shortage_id) REFERENCES drug_shortages(shortage_id)
);
"""

**3. The ETL Pipeline (Data Migration)**

We developed a Python-based ETL pipeline to manage the flow of data. This script utilizes `pandas` for data manipulation and `SQLAlchemy` for database interaction.

**3.1 Security & Configuration**

To adhere to industry best practices, we decoupled our configuration from our logic. We created a separate `config.py` file to store sensitive credentials (API Keys, Database Passwords). This ensures our code is secure and shareable without exposing secrets.

**3.2 Transformation Logic**

Our pipeline handles several critical data cleaning steps before loading:

- **Composite Key Generation:** Since the API lacks a consistent primary key, we generate a `shortage_id` by hashing the Drug Name and Manufacturer.

- **Date Standardization:** The API returns dates in US format (`MM/DD/YYYY`), which conflicts with PostgreSQL's ISO standard. We implemented `pd.to_datetime()` to automatically convert these strings into valid Date objects.

- **Case Sensitivity Handling:** We mapped all Python dictionary keys to lowercase (e.g., `drugName` -> `drugname`) to resolve conflicts with PostgreSQL's case-insensitive column naming convention.

**3.3 Loading Strategy**

We implemented a **"Refresh" strategy** for this phase. To ensure our dataset is always current and free of "Duplicate Key" errors during development, our script executes a `TRUNCATE ... CASCADE`command before loading. This wipes the tables clean while preserving the schema structure and indexes.

In [7]:
# Pipeline Code Snippet (Core Logic)

def transform_and_load(results):
    # ... (Logic to parse JSON) ...
    
    # Date Fix for PostgreSQL
    df_shortages['lastupdate'] = pd.to_datetime(df_shortages['lastupdate'], errors='coerce')

    # Atomic Transaction to Refresh Data
    with engine.begin() as conn:
        conn.execute(text(
            "TRUNCATE TABLE drug_shortages, affected_products RESTART IDENTITY CASCADE;"
        ))
    
    # Bulk Load
    df_shortages.to_sql('drug_shortages', con=engine, if_exists='append', index=False)
    df_products.to_sql('affected_products', con=engine, if_exists='append', index=False)

**4. Challenges & Solutions**

During the migration, we encountered and resolved two significant technical hurdles:

1. **Foreign Key Constraint Violations:**

- *Issue*: Attempting to drop or replace the parent table (`drug_shortages`) failed because the child table (`affected_products`) depended on it.

- *Solution*: We switched from a `DROP TABLE` approach to a `TRUNCATE TABLE ... CASCADE` approach, which respects the relationship constraints while still clearing the data.

2. **Date Format Overflow:**

- *Issue*: PostgreSQL threw a `DatetimeFieldOverflow` error when receiving `MM/DD/YYYY` formatted strings.

- **Solution**: We integrated a transformation step using Pandas to coerce all date strings into Python Datetime objects prior to SQL insertion.

**5. Conclusion**

We have successfully set up a robust, scalable data pipeline. The database is now populated with live data from OpenFDA, normalized for efficiency, and ready for the complex preprocessing and analysis tasks scheduled for Phase 3.

Github Link for the files: 

https://github.com/tagalogclark123-cmyk/Data-Mining-Phase-2-Repo.git